# Feature Engineering for Credit Risk Modeling

## Objective
The goal of this notebook is to transform raw borrower information into meaningful behavioral features that reflect repayment capacity and credit risk.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/raw/lendingclub.csv")

C:\Users\bloom\AppData\Local\Temp\ipykernel_21204\4086214078.py:1: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/raw/lendingclub.csv")


In [3]:
valid_status = ["Fully Paid", "Charged Off"]

df = df[df["loan_status"].isin(valid_status)].copy()

df["default"] = np.where(df["loan_status"] == "Charged Off", 1, 0)

In [4]:
df = df[
    [
        "loan_amnt",
        "term",
        "int_rate",
        "annual_inc",
        "dti",
        "fico_range_low",
        "fico_range_high",
        "revol_util",
        "emp_length",
        "default"
    ]
]


In [5]:
df["term"] = df["term"].str.extract(r"(\d+)").astype(int)

In [6]:
df["fico_avg"] = (df["fico_range_low"] + df["fico_range_high"]) / 2

df = df.drop(columns=["fico_range_low", "fico_range_high"])

In [7]:
df.loc[df["annual_inc"] == 0, "annual_inc"] = np.nan


In [8]:
df["loan_to_income"] = df["loan_amnt"] / df["annual_inc"]


In [9]:
df["revol_util"] = df["revol_util"].clip(upper=100)

In [10]:
df["revol_util"] = df["revol_util"].fillna(df["revol_util"].median())

In [11]:
df["emp_length"] = df["emp_length"].replace({
    "10+ years": 10,
    "< 1 year": 0,
    "1 year": 1,
    "2 years": 2,
    "3 years": 3,
    "4 years": 4,
    "5 years": 5,
    "6 years": 6,
    "7 years": 7,
    "8 years": 8,
    "9 years": 9
})

df["emp_length"] = pd.to_numeric(df["emp_length"])

C:\Users\bloom\AppData\Local\Temp\ipykernel_21204\1279707493.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["emp_length"] = df["emp_length"].replace({


In [12]:
df["annual_inc"] = df["annual_inc"].fillna(df["annual_inc"].median())
df["dti"] = df["dti"].fillna(df["dti"].median())
df["emp_length"] = df["emp_length"].fillna(df["emp_length"].median())
df["loan_to_income"] = df["loan_to_income"].fillna(df["loan_to_income"].median())

In [13]:
df.dtypes
df.isna().sum()
df.describe()

,loan_amnt,term,int_rate,annual_inc,dti,revol_util,emp_length,default,fico_avg,loan_to_income
count,1.345310e+06,1.345310e+06,1.345310e+06,1.345310e+06,1.345310e+06,1.345310e+06,1.345310e+06,1.345310e+06,1.345310e+06,1.345310e+06
mean,1.441997e+04,4.179020e+01,1.323962e+01,7.626508e+04,1.828248e+01,5.179824e+01,5.967865e+00,1.996261e-01,6.981851e+02,3.973044e-01
std,8.717051e+03,1.026832e+01,4.768716e+00,6.991418e+04,1.115890e+01,2.447533e+01,3.581850e+00,3.997195e-01,3.185284e+01,7.081920e+01
min,5.000000e+02,3.600000e+01,5.310000e+00,1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.270000e+02,1.714286e-04
25%,8.000000e+03,3.600000e+01,9.750000e+00,4.590000e+04,1.179000e+01,3.350000e+01,3.000000e+00,0.000000e+00,6.720000e+02,1.246903e-01
50%,1.200000e+04,3.600000e+01,1.274000e+01,6.500000e+04,1.761000e+01,5.220000e+01,6.000000e+00,0.000000e+00,6.920000e+02,2.000000e-01
75%,2.000000e+04,3.600000e+01,1.599000e+01,9.000000e+04,2.405000e+01,7.070000e+01,1.000000e+01,0.000000e+00,7.120000e+02,2.909091e-01
max,4.000000e+04,6.000000e+01,3.099000e+01,1.099920e+07,9.990000e+02,1.000000e+02,1.000000e+01,1.000000e+00,8.475000e+02,4.000000e+04


In [15]:
df.loc[df["dti"] < 0, "dti"] = df["dti"].median()

In [16]:
df["loan_to_income"] = df["loan_to_income"].clip(upper=10)

In [17]:
df.dtypes
df.isna().sum()
df.describe()

,loan_amnt,term,int_rate,annual_inc,dti,revol_util,emp_length,default,fico_avg,loan_to_income
count,1.345310e+06,1.345310e+06,1.345310e+06,1.345310e+06,1.345310e+06,1.345310e+06,1.345310e+06,1.345310e+06,1.345310e+06,1.345310e+06
mean,1.441997e+04,4.179020e+01,1.323962e+01,7.626508e+04,1.828251e+01,5.179824e+01,5.967865e+00,1.996261e-01,6.981851e+02,2.162308e-01
std,8.717051e+03,1.026832e+01,4.768716e+00,6.991418e+04,1.115888e+01,2.447533e+01,3.581850e+00,3.997195e-01,3.185284e+01,1.542117e-01
min,5.000000e+02,3.600000e+01,5.310000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.270000e+02,1.714286e-04
25%,8.000000e+03,3.600000e+01,9.750000e+00,4.590000e+04,1.179000e+01,3.350000e+01,3.000000e+00,0.000000e+00,6.720000e+02,1.246903e-01
50%,1.200000e+04,3.600000e+01,1.274000e+01,6.500000e+04,1.761000e+01,5.220000e+01,6.000000e+00,0.000000e+00,6.920000e+02,2.000000e-01
75%,2.000000e+04,3.600000e+01,1.599000e+01,9.000000e+04,2.405000e+01,7.070000e+01,1.000000e+01,0.000000e+00,7.120000e+02,2.909091e-01
max,4.000000e+04,6.000000e+01,3.099000e+01,1.099920e+07,9.990000e+02,1.000000e+02,1.000000e+01,1.000000e+00,8.475000e+02,1.000000e+01


In [18]:
df.to_csv("../data/processed/model_data.csv", index=False)

### Alignment with EDA

Feature selection and transformations were informed by EDA insights to ensure consistency between observed risk patterns and engineered behavioral features.